In [1]:
# Importaciones necesarias
import os
import requests
import pymysql
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from datetime import date
print('Setup Complete')

Setup Complete


In [2]:
# Carga las variables de entorno
load_dotenv()

# Lee las variables de entorno
host = os.getenv('DB_HOST')
user = os.getenv('DB_USERNAME')
password = os.getenv('DB_PASSWORD')

In [3]:
# Conexión a la base de datos MySQL
try:
    conn = pymysql.connect(host=host, user=user, password=password)
    cursor = conn.cursor()
    print("Conexión exitosa")
except Exception as e:
    print("Ocurrió un error al conectar:", e)

Conexión exitosa


In [4]:
# Ejecuta una consulta para crear la base de datos
try:
    cursor.execute('CREATE DATABASE IF NOT EXISTS cuspide')
    print("Base de datos creada con exito")
except Exception as e:
    print("Ocurrió un error al crear la base de datos:", e)

Base de datos creada con exito


In [5]:
try:
    # Usar la base de datos "cuspide"
    cursor.execute('USE cuspide')

    # Crea la tabla books
    query = """
    CREATE TABLE IF NOT EXISTS books (
        id_Libro INT AUTO_INCREMENT PRIMARY KEY,
        titulo VARCHAR(255),
        url VARCHAR(255),
        precio FLOAT,
        precio_usd FLOAT,
        precio_usd_blue FLOAT,
        fecha DATE
    )
    """

    cursor.execute(query)

    # Crea la tabla de auditoría de errores
    query = """
    CREATE TABLE IF NOT EXISTS errors (
        idError INT AUTO_INCREMENT PRIMARY KEY,
        titulo VARCHAR(255),
        url VARCHAR(255),
        fecha DATE,
        error VARCHAR(255)
    )
    """

    cursor.execute(query)

    print("Tablas creadas con exito")
except Exception as e:
    print("Ocurrió un error al crear las tablas:", e)

Tablas creadas con exito


In [6]:
# Obtiene el valor del dolar blue
try:
    url = 'https://dolarhoy.com/cotizaciondolarblue'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    dolar_blue = float(soup.find_all('div', class_='value')[1].get_text(strip=True).replace('$', ''))
    print('Valor dolar blue:', dolar_blue)
except Exception as e:
    print("Hay un problema con la solicitud:", e)

Valor dolar blue: 970.0


In [7]:
# Obtiene la página web de cuspide
try:
    url = 'https://cuspide.com/100-mas-vendidos/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    print("Solicitud a la página web de cuspide exitosa \nExtrayendo los datos de los libros:")
except Exception as e:
    print("Hay un problema con la solicitud.", e)

Solicitud a la página web de cuspide exitosa 
Extrayendo los datos de los libros:


In [8]:
# Busca todos los elementos <h3> que contienen el título y la URL de cada libro
books = soup.find_all(
    "h3", class_="name product-title woocommerce-loop-product__title")

In [9]:
# Crea una lista vacía para guardar los datos de los libros
libros = []

# Crea una lista vacía para guardar los errores
errores = []

In [10]:
# Itera sobre cada elemento <h3> y extraemos la URL y el título
for book in books:
    link = book.find("a")  
    title = link.text  
    url = link["href"]  

    # Obtiene la página web con la descripción de cada libro
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        p = soup.find("p", class_="price product-page-price")
        span1 = p.find('span', string='$').next_sibling
        span2 = soup.find("span", style="font-size: 1.3em")
        # Obtiene el precio del libro en pesos y dolares
        precio = float(span1.get_text(strip=True).replace('.', '').replace(',', '.'))
        precio_usd = float(span2.get_text(strip=True).replace('.', '').replace(',', '.'))
        # Calcula el precio del libro en dolares blue
        precio_usd_blue = round(precio/dolar_blue, 2)
        # Obtiene la fecha actual
        fecha = date.today().isoformat()
        # Crea un diccionario con los datos del libro
        libro = {
            "titulo": title,
            "url": url,
            "precio": precio,
            "precio_usd": precio_usd,
            "precio_usd_blue": precio_usd_blue,
            "fecha": fecha
        }
        # Agrega el diccionario del libro a la lista de libros
        libros.append(libro)
        print(libro)
    except Exception as e:
        print(f"Ocurrió un error al extraer los datos del libro {title}:", e)
        # Si ocurre algún error, crear un diccionario con el error
        error = {
            "titulo": title,
            "url": url,
            "fecha": fecha,
            "error": e
        }
        # Agrega el diccionario del error a la lista de errores
        errores.append(error)

{'titulo': 'LA FELICIDAD', 'url': 'https://cuspide.com/producto/la-felicidad-2/', 'precio': 16900.0, 'precio_usd': 20.58, 'precio_usd_blue': 17.42, 'fecha': '2023-12-16'}
{'titulo': 'HEARTSTOPPER 5', 'url': 'https://cuspide.com/producto/heartstopper-5/', 'precio': 8249.0, 'precio_usd': 10.05, 'precio_usd_blue': 8.5, 'fecha': '2023-12-16'}
{'titulo': 'DESTROZA ESTE DIARIO ( A TODO COLOR )', 'url': 'https://cuspide.com/producto/destroza-este-diario-a-todo-color/', 'precio': 20200.0, 'precio_usd': 24.6, 'precio_usd_blue': 20.82, 'fecha': '2023-12-16'}
{'titulo': 'ZENSORIALMENTE', 'url': 'https://cuspide.com/producto/zensorialmente/', 'precio': 10999.0, 'precio_usd': 13.4, 'precio_usd_blue': 11.34, 'fecha': '2023-12-16'}
{'titulo': 'COMO SALIR DEL POZO', 'url': 'https://cuspide.com/producto/como-salir-del-pozo/', 'precio': 9999.0, 'precio_usd': 12.18, 'precio_usd_blue': 10.31, 'fecha': '2023-12-16'}
{'titulo': 'ALAS DE SANGRE', 'url': 'https://cuspide.com/producto/alas-de-sangre/', 'precio

In [11]:
# Imprime la lista de libros
print("Libros:")
for libro in libros:
    print(f"Título: {libro['titulo']}")
    print(f"URL: {libro['url']}")
    print(f"AR$: {libro['precio']}")
    print(f"U$s: {libro['precio_usd']}")
    print(f"U$s Blue: {libro['precio_usd_blue']}")
    print(f"Fecha: {libro['fecha']}")
    print()

Libros:
Título: LA FELICIDAD
URL: https://cuspide.com/producto/la-felicidad-2/
AR$: 16900.0
U$s: 20.58
U$s Blue: 17.42
Fecha: 2023-12-16

Título: HEARTSTOPPER 5
URL: https://cuspide.com/producto/heartstopper-5/
AR$: 8249.0
U$s: 10.05
U$s Blue: 8.5
Fecha: 2023-12-16

Título: DESTROZA ESTE DIARIO ( A TODO COLOR )
URL: https://cuspide.com/producto/destroza-este-diario-a-todo-color/
AR$: 20200.0
U$s: 24.6
U$s Blue: 20.82
Fecha: 2023-12-16

Título: ZENSORIALMENTE
URL: https://cuspide.com/producto/zensorialmente/
AR$: 10999.0
U$s: 13.4
U$s Blue: 11.34
Fecha: 2023-12-16

Título: COMO SALIR DEL POZO
URL: https://cuspide.com/producto/como-salir-del-pozo/
AR$: 9999.0
U$s: 12.18
U$s Blue: 10.31
Fecha: 2023-12-16

Título: ALAS DE SANGRE
URL: https://cuspide.com/producto/alas-de-sangre/
AR$: 21200.0
U$s: 25.82
U$s Blue: 21.86
Fecha: 2023-12-16

Título: EL PSICOANALISTA EN LA MIRA
URL: https://cuspide.com/producto/el-psicoanalista-en-la-mira/
AR$: 15999.0
U$s: 19.49
U$s Blue: 16.49
Fecha: 2023-12-16

In [12]:
# Imprime la lista de errores
print("Errores:")
for error in errores:
    print(f"Título: {error['titulo']}")
    print(f"URL: {error['url']}")
    print(f"Error: {error['error']}")
    print(f"Fecha: {error['fecha']}")
    print()

Errores:
Título: LO MEJOR Y LO PEOR DE INTERNET
URL: https://cuspide.com/producto/lo-mejor-y-lo-peor-de-internet/
Error: 'NoneType' object has no attribute 'find'
Fecha: 2023-12-16

Título: HABITOS ATOMICOS
URL: https://cuspide.com/producto/habitos-atomicos-2/
Error: 'NoneType' object has no attribute 'find'
Fecha: 2023-12-16

Título: LOS JUEGOS DEL HAMBRE ( LIBRO 1 DE LA SAGA LOS JUEGOS DEL HAMBRE )
URL: https://cuspide.com/producto/los-juegos-del-hambre-libro-1-de-la-saga-los-juegos-del-hambre/
Error: 'NoneType' object has no attribute 'find'
Fecha: 2023-12-16

Título: EL SUTIL ARTE DE QUE ( CASI TODO ) TE IMPORTE UNA MIERDA
URL: https://cuspide.com/producto/el-sutil-arte-de-que-casi-todo-te-importe-una-mierda/
Error: 'NoneType' object has no attribute 'find'
Fecha: 2023-12-16

Título: TAYLOR : FROM THE VAULT
URL: https://cuspide.com/producto/taylor-from-the-vault/
Error: 'NoneType' object has no attribute 'find'
Fecha: 2023-12-16

Título: SI LO CREES , LO CREAS
URL: https://cuspide.

In [13]:
# Itera sobre la lista de libros
try:
    print("Ingestando los datos de los libros en la base de datos...")
    for libro in libros:
        # Crea la consulta SQL para insertar los datos del libro
        query = f"""
        INSERT INTO books (titulo, url, precio, precio_usd, precio_usd_blue, fecha)
        VALUES ("{libro["titulo"]}", "{libro["url"]}", {libro["precio"]}, {libro["precio_usd"]}, {libro["precio_usd_blue"]}, "{libro["fecha"]}")
        """
        # Ejecuta la consulta
        cursor.execute(query)

    # Itera sobre la lista de errores
    for error in errores:
        # Crea la consulta SQL para insertar los datos del error
        query = f"""
        INSERT INTO errors (titulo, url, fecha, error)
        VALUES ("{error["titulo"]}", "{error["url"]}", "{error["fecha"]}", "{error["error"]}")
        """
        # Ejecuta la consulta
        cursor.execute(query)

    # Confirma los cambios
    conn.commit()
    print("Ingesta de datos finalizada con éxito")
    
except Exception as e:
    # Manejo de errores
    print("Ocurrió un error:", e)


Ingestando los datos de los libros en la base de datos...
Ingesta de datos finalizada con éxito


In [16]:
try:
    # Selecciona todos los registros de la tabla books
    cursor.execute("SELECT * FROM books")
    libros = cursor.fetchall()
    print("Datos de la tabla books:")
    for libro in libros:
        print(libro)

    # Selecciona todos los registros de la tabla errors
    cursor.execute("SELECT * FROM errors")
    errores = cursor.fetchall()
    print("Datos de la tabla errors:")
    for error in errores:
        print(error)

except Exception as e:
    # Manejo de errores
    print("Ocurrió un error:", e)

finally:
    # Cerrar la conexión
    if conn:
        cursor.close()
        conn.close()
        print("Conexión cerrada")


Conexión exitosa
Datos de la tabla books:
(1, 'LA FELICIDAD', 'https://cuspide.com/producto/la-felicidad-2/', 16900.0, 20.58, 17.42, datetime.date(2023, 12, 16))
(2, 'HEARTSTOPPER 5', 'https://cuspide.com/producto/heartstopper-5/', 8249.0, 10.05, 8.5, datetime.date(2023, 12, 16))
(3, 'DESTROZA ESTE DIARIO ( A TODO COLOR )', 'https://cuspide.com/producto/destroza-este-diario-a-todo-color/', 20200.0, 24.6, 20.82, datetime.date(2023, 12, 16))
(4, 'ZENSORIALMENTE', 'https://cuspide.com/producto/zensorialmente/', 10999.0, 13.4, 11.34, datetime.date(2023, 12, 16))
(5, 'COMO SALIR DEL POZO', 'https://cuspide.com/producto/como-salir-del-pozo/', 9999.0, 12.18, 10.31, datetime.date(2023, 12, 16))
(6, 'ALAS DE SANGRE', 'https://cuspide.com/producto/alas-de-sangre/', 21200.0, 25.82, 21.86, datetime.date(2023, 12, 16))
(7, 'EL PSICOANALISTA EN LA MIRA', 'https://cuspide.com/producto/el-psicoanalista-en-la-mira/', 15999.0, 19.49, 16.49, datetime.date(2023, 12, 16))
(8, 'ESTE DOLOR NO ES MIO', 'https